In [72]:
!pip install vowpalwabbit


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
import vowpalwabbit
import datetime


In [74]:
!sudo apt-get install vowpal-wabbit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
vowpal-wabbit is already the newest version (8.5.0.dfsg1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [75]:
import os
import numpy as np
import re
from sklearn.datasets import load_files
from sklearn.metrics import accuracy_score, roc_auc_score
import json
import datetime


In [76]:
def to_vw_format(document, label=None):
      return str(label or '') + ' |text ' + ' '.join(re.findall('\w{3,}', document.lower())) + '\n'

In [77]:
read_start = datetime.datetime.now()

with open('combined_names_newest.json') as f:
    dataset = json.load(f)
    

# NB: this is getting all used pt_ids, also reformatting in something more like what we might use
pts = set()
for item in dataset:
    item_pts = []
    # NB: this replaces the provision_type_ids list;
    # the file has it as a list of strings like this: "label-true/false"
    # we reformat the list so it is a list of tuples: ['label', bool]
    for label in item['labels']['provision_type_ids']:
        l = label.rsplit('-', 1)
        if l[1] == 'true':
            l[1] = 1
        else:
            l[1] = 0
        pts.add(l[0])
        item_pts.append(l)
        #if l[0] == 'Governing Law':
        #     pts.add(l[0])
        #     item_pts.append(l)
    item['labels']['provision_type_ids'] = item_pts


read_end = datetime.datetime.now()
print('file load:', (read_end - read_start))

file load: 0:00:02.980018


In [78]:
train_size = int(0.7 * len(dataset))


In [79]:
new_list = [] #all provision tags with labels
for item in dataset:

  for key,value in item['labels'].items():
    new_list.append(value)

In [80]:
final_list = [] #only first provision tag
for i in range(len(new_list)):
  final_list.append(new_list[i][63])

In [81]:
labels_new = [] #only outcome of the provision tag if 1 or 0
for i in range(len(final_list)):
  labels_new.append(final_list[i][-1])

In [82]:
len(labels_new) #final labels

18051

In [83]:
y_train = labels_new[:train_size] #splitting final labels

In [84]:
# train = []
# for item in dataset:
#   train.append(item['features'])

In [85]:
# new_train = []
# for i in train:
#     new_train.append(i)

In [86]:
# text = {}
# for i in range(len(new_train)):
#   b = new_train[i]
#   text.update(b)

In [87]:
# text

In [88]:
final_text = [] #core_lemma_text and compound core lemma text splitted
for item in dataset[:train_size]:
  for key,value in item['features'].items():
    final_text.append(value)

In [89]:
len(final_text)

25270

In [90]:
del final_text[1::2]

In [91]:
len(final_text) #final text with only core_lemma_text

12635

In [92]:
read_start = datetime.datetime.now()


with open('/content/combined_names_newest.vw', 'w') as vw_train_data:
   
   for text, target in zip(final_text,y_train):
     vw_train_data.write(to_vw_format(str(text), 1 if target == 1 else -1))


read_end = datetime.datetime.now()
print('time taken:', (read_end - read_start))

time taken: 0:00:00.139082


In [93]:
! head -200 combined_names_newest.vw

-1 |text delivery amount return amount credit support amount
-1 |text aaa1 aa3p1 moody
-1 |text maturity eight year
-1 |text equivalent substitute
-1 |text resolution time mean london time local business day follow date notice give give rise dispute paragraph
-1 |text royal bank canada
-1 |text valuation date
-1 |text negotiable debt obligation issue government united
-1 |text finland minimum rating
-1 |text time time
-1 |text close business place location valuation agent valuation date date calculation applicable
-1 |text close business local business day immediately precede valuation date date calculation applicable
-1 |text exchange
-1 |text attn collateral management
-1 |text finland minimum rating
-1 |text aaa1 aa3p1 moody
-1 |text valuation timing
-1 |text party
-1 |text 2nd floor south tower
-1 |text 
-1 |text negotiable debt obligation issue government united
-1 |text agency have remain
-1 |text threshold mean respect party zero
-1 |text minimum transfer amount mean respect par

In [94]:
# new_list = []
# for item in dataset:

#   for key,value in item['labels'].items():
#     new_list.append(value)

In [95]:
# labels_new = []
# for j in range(len(new_list)):  
#   for i in range(len(labels)):
#     new = []
#     new = labels[i]

#     labels_new.append(new[-1])

In [96]:
!vw -d /content/combined_names_newest.vw --loss_function logistic -f model2.vw

final_regressor = model2.vw
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /content/combined_names_newest.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.693147 0.693147            1            1.0  -1.0000   0.0000        8
0.661258 0.629368            2            2.0  -1.0000  -0.1319        4
0.608957 0.556656            4            4.0  -1.0000  -0.3431        3
0.536290 0.463622            8            8.0  -1.0000  -0.5855        7
0.408241 0.280193           16           16.0  -1.0000  -1.3616        4
0.317998 0.227754           32           32.0  -1.0000  -2.1592        4
0.211691 0.105385           64           64.0  -1.0000  -2.5546        5
0.138000 0.064309          128          128.0  -1.0000  -4.5460        8
0.087135 0.036269          256          256.0  -1.0000  -3.0254        4
0.054847 0.02255

In [97]:
y_test = labels_new[train_size:] #test lable spliitted

In [98]:
final_text_test = [] #test splitted along with core_lemma_text and compound_core_lemma_text
for item in dataset[train_size:]:
  for key,value in item['features'].items():
    final_text_test.append(value)

In [99]:
len(final_text_test)

10832

In [100]:
del final_text_test[1::2] #test splitted only core_lemma_text

In [101]:
len(final_text_test)

5416

In [102]:
with open('/content/combined_names_newest_test.vw', 'w') as vw_train_data:
   for text, target in zip(final_text_test,y_test):
     vw_train_data.write(to_vw_format(str(text), 1 if target == 1 else -1))

In [103]:
! head -200 combined_names_newest_test.vw

-1 |text purpose section agreement party agree deliver follow document applicable
-1 |text copy most recent annual financial report certified independent public accountant
-1 |text telephone 33416000
-1 |text part tax representation
-1 |text documentation evidence authority incumbency specimen signature person execute document other document relate thereto
-1 |text 
-1 |text party
-1 |text 
-1 |text adaptivelending
-1 |text facsimile
-1 |text process agent purpose section 13c agreement
-1 |text signing agreement
-1 |text address
-1 |text credit support provider mean relation party applicable
-1 |text govern law agreement govern construe accordance english law
-1 |text storebæltsforbindelsen
-1 |text specify entity mean relation party purpose
-1 |text section 5biv applicable
-1 |text second method apply
-1 |text payer tax representation applicable
-1 |text execution delivery agreement
-1 |text adaptive lane 2010
-1 |text address
-1 |text credit support provider credit support provider m

In [104]:
!vw -i model2.vw -t -d /content/combined_names_newest_test.vw -p final.txt 


only testing
predictions = final.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /content/combined_names_newest_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
106.418480 106.418480            1            1.0  -1.0000 -11.3159       10
105.981236 105.543991            2            2.0  -1.0000 -11.2735       11
79.350600 52.719964            4            4.0  -1.0000 -10.4887        4
73.778350 68.206101            8            8.0  -1.0000  -4.2905        1
56.419221 39.060091           16           16.0  -1.0000  -4.5714        2
49.318188 42.217155           32           32.0  -1.0000  -7.6205        3
66.830224 84.342260           64           64.0  -1.0000  -8.6956        8
101.899987 136.969750          128          128.0  -1.0000   1.5449       39
108.913956 115.927925          256          256.0  -1

In [107]:
with open('final.txt') as pred_file:
     valid_prediction = [float(label) for label in pred_file.readlines()]
     print("Accuracy: {}".format(round(accuracy_score(y_test, [int(pred_prob > 0) for pred_prob in valid_prediction]), 5)))
     print("AUC: {}".format(round(roc_auc_score(y_test, valid_prediction), 5)))


Accuracy: 0.99649
AUC: 0.97943


Accuracy: 0.99649
AUC: 0.97943 
on core_lemma_text